# Visualize MuData in Vitessce

In [1]:
!lamin load laminlabs/lamindata

💡 connected lamindb: laminlabs/lamindata


In [2]:
import lamindb as ln

ln.context.uid = "aCrfeVoAaDr90000"
ln.context.track()

💡 connected lamindb: laminlabs/lamindata


💡 notebook imports: lamindb==0.70.3 mudata==0.2.3 scanpy==1.10.1 vitessce==3.2.5
💡 saved: Transform(uid='aCrfeVoAaDr95zKv', name='Visualize MuData in Vitessce', key='mudata-vitessce', version='1', type='notebook', updated_at=2024-04-30 12:46:10 UTC, created_by_id=2)
💡 saved: Run(uid='VPYyibO30e6mHkw7DC7r', transform_id=82, created_by_id=2)


## Prepare dataset

Load MuData object from: https://lamin.ai/laminlabs/lamindata/artifact/DDFxKYXoNMmHzQAHqpx0

In [3]:
mdata_artifact = ln.Artifact.get("DDFxKYXo")
mdata = mdata_artifact.load()
mdata

/opt/miniconda3/envs/py310/lib/python3.10/site-packages/anndata/_core/anndata.py:430: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/opt/miniconda3/envs/py310/lib/python3.10/site-packages/anndata/_core/anndata.py:430: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/opt/miniconda3/envs/py310/lib/python3.10/site-packages/anndata/_core/anndata.py:430: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/opt/miniconda3/envs/py310/lib/python3.10/site-packages/anndata/_core/anndata.py:430: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/opt/miniconda3/envs/py310/lib/python3.10/site-packages/mudata/_core/mudata.py:491: UserWarning: Cannot join columns with the same name because var_names are intersecting.
  warnings.warn(


MuData object with n_obs × n_vars = 20729 × 18776
  4 modalities
    rna:	20729 x 18649
      obs:	'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_HTO', 'nFeature_HTO', 'nCount_GDO', 'nCount_ADT', 'nFeature_ADT', 'percent.mito', 'MULTI_ID', 'HTO_classification', 'guide_ID', 'gene_target', 'NT', 'perturbation', 'replicate', 'S.Score', 'G2M.Score', 'Phase'
      var:	'name', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
      uns:	'Phase_colors', 'hvg', 'log1p', 'neighbors', 'pca', 'perturbation_colors', 'replicate_colors', 'umap'
      obsm:	'X_pca', 'X_umap'
      varm:	'PCs'
      obsp:	'connectivities', 'distances'
    adt:	20729 x 4
      obs:	'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_HTO', 'nFeature_HTO', 'nCount_GDO', 'nCount_ADT', 'nFeature_ADT', 'percent.mito', 'MULTI_ID', 'HTO_classification', 'guide_ID', 'gene_target', 'NT', 'perturbation', 'replicate', 'S.Score', 'G2M.Score', 'Phase'
      var:	'name'
    hto:	20729 x 12
      obs:	'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_HTO', 'nFeature_HTO', 'nCount_GDO', 'nCount_ADT', 'nFeature_ADT', 'percent.mito', 'MULTI_ID', 'HTO_classification', 'guide_ID', 'gene_target', 'NT', 'perturbation', 'replicate', 'S.Score', 'G2M.Score', 'Phase'
      var:	'name'
      uns:	'neighbors', 'umap'
      obsm:	'X_umap'
      obsp:	'connectivities', 'distances'
    gdo:	20729 x 111
      obs:	'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_HTO', 'nFeature_HTO', 'nCount_GDO', 'nCount_ADT', 'nFeature_ADT', 'percent.mito', 'MULTI_ID', 'HTO_classification', 'guide_ID', 'gene_target', 'NT', 'perturbation', 'replicate', 'S.Score', 'G2M.Score', 'Phase'
      var:	'name'

Subset to hvg for visualization:

In [4]:
mdata.mod["rna"] = mdata["rna"][:, mdata["rna"].var["highly_variable"]]
mdata.update()

/opt/miniconda3/envs/py310/lib/python3.10/site-packages/mudata/_core/mudata.py:491: UserWarning: Cannot join columns with the same name because var_names are intersecting.
  warnings.warn(


Write to `anndata.zarr`:

In [5]:
output_path = "mudata_papalexi21.anndata.zarr"
mdata.write_zarr(output_path)

/opt/miniconda3/envs/py310/lib/python3.10/site-packages/mudata/_core/mudata.py:491: UserWarning: Cannot join columns with the same name because var_names are intersecting.
  warnings.warn(


## Save dataset

In [6]:
zarr_artifact = ln.Artifact(
    output_path,
    description="MuData from Papalexi21",
)
zarr_artifact.save()

In [7]:
from vitessce import (
    VitessceConfig,
    Component as cm,
    AnnDataWrapper,
)

vc = VitessceConfig(
    schema_version="1.0.15",
    description=zarr_artifact.description,
)
dataset = vc.add_dataset(name="Papalexi21").add_object(
    AnnDataWrapper(
        adata_url=zarr_artifact.path.to_url(),
        obs_feature_matrix_path="mod/rna/X",
        obs_embedding_paths=["mod/rna/obsm/X_umap", "mod/rna/obsm/X_pca"],
        obs_embedding_names=["UMAP", "PCA"],
        obs_set_paths=[
            "mod/rna/obs/gene_target",
            "mod/rna/obs/guide_ID",
            "mod/rna/obs/Phase",
            "mod/rna/obs/replicate",
        ],
        obs_set_names=["Gene target", "Guide id", "Cell cycle", "Replicate"],
    )
)

obs_sets = vc.add_view(cm.OBS_SETS, dataset=dataset)
obs_sets_sizes = vc.add_view(cm.OBS_SET_SIZES, dataset=dataset)
umap = vc.add_view(cm.SCATTERPLOT, dataset=dataset, mapping="UMAP")
pca = vc.add_view(cm.SCATTERPLOT, dataset=dataset, mapping="PCA")
heatmap = vc.add_view(cm.HEATMAP, dataset=dataset)
genes = vc.add_view(cm.FEATURE_LIST, dataset=dataset)
vc.layout(((umap | pca) / heatmap) | ((genes | obs_sets) / obs_sets_sizes))

# inspect the config
vc.to_dict()

{'version': '1.0.15',
 'name': '',
 'description': 'MuData from Papalexi21',
 'datasets': [{'uid': 'A',
   'name': 'Papalexi21',
   'files': [{'fileType': 'anndata.zarr',
     'url': 'https://lamindata.s3.amazonaws.com/.lamindb/tCUkRcaEjTjhtozp.anndata.zarr',
     'options': {'obsEmbedding': [{'path': 'mod/rna/obsm/X_umap',
        'dims': [0, 1],
        'embeddingType': 'UMAP'},
       {'path': 'mod/rna/obsm/X_pca', 'dims': [0, 1], 'embeddingType': 'PCA'}],
      'obsSets': [{'name': 'Gene target', 'path': 'mod/rna/obs/gene_target'},
       {'name': 'Guide id', 'path': 'mod/rna/obs/guide_ID'},
       {'name': 'Cell cycle', 'path': 'mod/rna/obs/Phase'},
       {'name': 'Replicate', 'path': 'mod/rna/obs/replicate'}],
      'obsFeatureMatrix': {'path': 'mod/rna/X'}}}]}],
 'coordinationSpace': {'dataset': {'A': 'A'},
  'embeddingType': {'A': 'UMAP', 'B': 'PCA'}},
 'layout': [{'component': 'obsSets',
   'coordinationScopes': {'dataset': 'A'},
   'x': 9.0,
   'y': 0.0,
   'w': 3.0,
   'h':

In [8]:
from lamindb.integrations import save_vitessce_config

vc_artifact = save_vitessce_config(vc, description="View Papalexi21 in Vitessce")

💡 go to: https://lamin.ai/laminlabs/lamindata/artifact/sRapK07mMtToihzFeTaf


In [11]:
ln.finish()

✅ cell execution numbers increase consecutively
✅ saved transform.source_code: Artifact(uid='j6zQfam6cVBHlOR1dTde', suffix='.ipynb', description='Source of transform aCrfeVoAaDr95zKv', version='1', size=4511, hash='cmD2CwZCSebUjDXlzngfmQ', hash_type='md5', visibility=0, key_is_virtual=True, updated_at=2024-04-30 12:52:46 UTC, storage_id=2, created_by_id=2)
✅ saved run.environment: Artifact(uid='nYO2R3tM4rhqB7mZHZnl', suffix='.txt', description='requirements.txt', size=10821, hash='Iu7Isnon8g7iS-XUyNFD7w', hash_type='md5', visibility=0, key_is_virtual=True, updated_at=2024-04-30 12:52:47 UTC, storage_id=2, created_by_id=2)
✅ saved transform.latest_report: Artifact(uid='NEuTTW4HAIX9ORtCuDOe', suffix='.html', description='Report of run VPYyibO30e6mHkw7DC7r', size=293911, hash='ADqBuImn9seir0EPXhGOcg', hash_type='md5', visibility=0, key_is_virtual=True, updated_at=2024-04-30 12:52:48 UTC, storage_id=2, created_by_id=2)
✅ go to: https://lamin.ai/laminlabs/lamindata/transform/aCrfeVoAaDr95zK